<a href="https://colab.research.google.com/github/jusimon/AlternusVera/blob/master/Common/FakeDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
googleDrive = GoogleDrive(gauth)
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
Colnames = ['Json_Id', 'Label', 'Statement', 'Subject', 'SpeakerName',
           'JobTitle', 'StateInfo', 'partyAffiliation',
           'BarelyTrueCounts', 'FalseCounts','HalfTrueCounts',
           'MostlyTrueCounts','PantsOnFireCounts','Context']

valid_link = "https://drive.google.com/open?id=1gaS9R-PQvER5DE1dAgh7z6v4u_euIvrO"
fluff, id3 = valid_link.split('=')
valid_filename = 'valid.tsv'
downloaded = googleDrive.CreateFile({'id':id3}) 
downloaded.GetContentFile(valid_filename)
valid_news = pd.read_csv(valid_filename, sep='\t', names = Colnames,error_bad_lines=False)

In [0]:
Factors = [
           {
               "name" : "Political_Affiliation",
               "git" : "https://github.com/jusimon/AlternusVera",
               "folder" :"AlternusVera",
               "file" : "Political_Affiliation",
               "model" : "/content/gdrive/My Drive/Drifters/Models/Politcal_Affiliation_Model.pkl"
               }   
           ]

In [4]:
!rm -rf AlternusVera
!git clone https://github.com/jusimon/AlternusVera
!cp AlternusVera/Political_Affiliation.py ./

Cloning into 'AlternusVera'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 129 (delta 23), reused 18 (delta 6), pack-reused 79
Receiving objects: 100% (129/129), 34.59 MiB | 19.76 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [5]:
!rm -rf AlternusVera
!git clone https://github.com/psaini79/AlternusVera
!cp AlternusVera/sensational.py ./

Cloning into 'AlternusVera'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 103 (delta 13), reused 12 (delta 4), pack-reused 77
Receiving objects: 100% (103/103), 98.34 MiB | 30.15 MiB/s, done.
Resolving deltas: 100% (38/38), done.
Checking out files: 100% (11/11), done.


In [0]:
from Political_Affiliation import Political_Affiliation
from sensational import sensational

##### This function calculates fakeness percentage using the weighted_percentage function passing all factors fakeness score. It calls respective factors class to get the fakeness score of individual factor

In [0]:
from gensim.models.doc2vec import Doc2Vec
def fakecheck(dataFrame):
  # biasedFactor = Biased_Factor(dataFrame).predict()
  PoliticalAff = Political_Affiliation(valid_news).predict()
  #  Authenticity = Authenticity(dataFrame).predict()
  # Sensationalism = sensational(valid_news).predict()
  # print('sensationalism_factor:',sensationalism)    
  # print('political_affilication_factor:',PoliticalAff)
  #  print('spelling_error_factor:',spelling_error_factor)
  #  print('spam_detection_factor:',spam_detection_factor)
  #  print('stance_detection_factor:',stance_detection_factor)
  return weighted_percentage(0, PoliticalAff, 0, 0)

In [0]:
def weighted_percentage(SensationalismF, politicalAffilitaionF, BiasedF, AuthenticityF):                       
    SensationalismF_Weigth=.84
    BiasedF_Weight=.53
    politicalAffilitaionF_Weight=.65
    AuthenticityF_Weight=.75    
    total_weight=SensationalismF_Weigth+BiasedF_Weight+politicalAffilitaionF_Weight+AuthenticityF_Weight
    return round((((SensationalismF*SensationalismF_Weigth)+
            (politicalAffilitaionF*politicalAffilitaionF_Weight)+
            (BiasedF*BiasedF_Weight)+
            (AuthenticityF*AuthenticityF_Weight))/total_weight)*100,2)

In [9]:
print(weighted_percentage(1,1,1,1))
print(weighted_percentage(0,0,0,0))
print(weighted_percentage(.8,.6,1,.7))

100.0
0.0
76.43


#### Check Fakeness of a news article

In [14]:
input_news='Obamacare will provide insurance to all non-U.S. residents, even if they are here illegally.A strong bipartisan majority in the House of Representatives voted to defund Obamacare.07732584351 - Rodger Burns - MSG = We tried to call you re your reply to our sms for a free nokia mobile + free camcorder. Please call now 08000930705 for delivery tomorrow.Amazon Enters eBook Subscription Market with Kindle Unlimited'
fakecheck(input_news)

calling political factor:
calling sensationalism factor:


12.24